In [1]:
from sqlalchemy import create_engine
import pymysql, pandas as pd
pymysql.install_as_MySQLdb()
import MySQLdb,pyarrow

### COMPANY INFO  - 우선순위 1번째

In [2]:
company=pd.read_pickle('data/company_true.pick')
print(company.shape)

(458268, 5)


### Brand data - 우선순위 2번째

In [5]:
brandCode = pd.read_excel('BRANDCODE.xlsx')
brandCode = brandCode.fillna('{null}')
brandCode.head(4)

,코드그룹,코드그룹명,코드,코드명,동의어,불용어
0,1,영화,0,CGV,^CGV[^점],투썸|스타벅스|이디야|엔제리너스|커피|카페|할리스|파스쿠찌|탐앤탐
1,1,영화,1,롯데시네마,롯데시네마[^점],투썸|스타벅스|이디야|엔제리너스|커피|카페|할리스|파스쿠찌|탐앤탐|망고|갈릭|브레드...
2,1,영화,2,메가박스,메가박스[^점]|megabox,투썸|스타벅스|이디야|엔제리너스|커피|카페|할리스|파스쿠찌|탐앤탐|pc|피씨|올리브영
3,1,영화,999,영화기타,^(이봄씨어터|씨네드쉐프|시네마파크|아트시네마|아트나인|상상마당시네마|산마을극장|필...,{null}


In [22]:
def lowerNospace(row):
    import re
    return re.sub(r'\s','',row.lower())

test = company[['COMPANY_ID','CO_NAME','CATE','CATE1']]
test = test.assign(CO_NAME = test.CO_NAME.apply(lowerNospace))


for index, obj in brandCode.iterrows():
    value = lowerNospace(obj['동의어'])
    if obj['불용어'] =='{null}': 
        test.loc[test.CO_NAME.str.contains(value),'BRAND']=  obj['코드명']
#         test[obj['코드명']]= test.CO_NAME.str.contains(value).astype(int)
    else:
        test.loc[~test.CO_NAME.str.contains(obj['불용어'])&test.CO_NAME.str.contains(value),'BRAND']=  obj['코드명']
#         test[obj['코드명']]= (~test.CO_NAME.str.contains(obj['불용어'])&test.CO_NAME.str.contains(value)).astype(int)


# 675개
test.loc[(test['BRAND'].isnull())&(test['CATE1']=="화장품"),'BRAND']='뷰티기타'
#22744
test.loc[(test['BRAND'].isnull())&(test['CATE1']=="카페"),'BRAND']='커피기타'

C:\Users\epopc\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


In [23]:
test[~test['BRAND'].isnull()].shape

(34189, 5)

In [24]:
test[test['BRAND'].isin(['뷰티기타','커피기타'])].shape

(23612, 5)

In [33]:
test=test.fillna('na')#[test['BRAND'].str.contains('기타')].head()
print(test[test['BRAND'].str.contains('기타')].shape)
test[test['BRAND'].str.contains('기타')].head()

(23620, 5)


,COMPANY_ID,CO_NAME,CATE,CATE1,BRAND
56,3088,다경,카페/디저트,카페,커피기타
77,3788,동빙고,카페/디저트,카페,커피기타
184,7236,아모제푸드,카페/디저트,카페,커피기타
293,10454,르돌치일구사육(주),카페/디저트,카페,커피기타
435,14951,1940나한정역커피가게,카페/디저트,카페,커피기타


In [35]:
test.to_pickle('data/company_true.pick')